# APEx STAC catalog integration with openEO

The project catalog provided by APEx can be used in combination with an openEO based platform.

Do note that openEO is an open standard, and not all platforms may support all use cases. Also the produced STAC metadata may have differences that are relevant to your project.
Therefore we always recommend to check the envisioned use cases with your provider of choice.

## Supported use cases

1. Scripted upload of openEO generated results into APEx catalog (supported by any openEO instance).
2. Direct publishing of results to object storage (requires support for openEO 'workspace' API).
3. Direct publishing of results to STAC catalogue (requires support for openEO 'workspace' API, and specifically merging into existing catalog.)

## Use case 1: Scripted upload

This use case is most broadly supported because it simply takes STAC metadata of openEO job results, downloads it, and then ingests it into a catalog.

The most important drawbacks of this method:

- Data is first downloaded from openEO backend and then uploaded again, which can take time and bandwidth, and may even be interrupted when facing network instability.
- Extra project specific code is needed to perform the task.

